[Reference](https://www.kaggle.com/bugraokcu/cnn-with-keras}) <br>
[Problem](https://www.kaggle.com/zalando-research/fashionmnist)

In [1]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data_train = pd.read_csv('./data/fashion-mnist_train.csv')
data_test = pd.read_csv('./data/fashion-mnist_test.csv')

In [3]:
data_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
img_rows = img_cols = 28
input_shape = (img_rows, img_cols, 1)

X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0])) # it's one hot
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_test /= 255
X_val /= 255

### 1. Raw Code

In [7]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1) # random as normal distribution but only in 2SD
    return tf.Variable(init_random_dist)
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

# conv func
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name='conv2d')
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# layer func
def convolutional_layer(input_x, shape):
    # [Height, Weight, Channel(l-1), Channel(l)]
    W = init_weights(shape)
    b = init_bias([shape[3]]) # shape[3] -> current # of channels
    return tf.nn.relu( conv2d(input_x, W) + b)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return (tf.matmul(input_layer, W) + b)

# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

# initiate layer
convo_1 = convolutional_layer(x,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling, [6, 6, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])

full = normal_full_layer(convo_2_flat, 1024)
full_layer_one = tf.nn.relu(full)

# placeholder
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout, 10) # output 0-9

# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()

# Session
batch_size = 50
steps = 500
sess = tf.InteractiveSession()
sess.run(init)

for i in range(steps):
    batch_x = X_train[steps*batch_size: steps*(batch_size+1)]
    batch_y = y_train[steps*batch_size: steps*(batch_size+1)]
    sess.run(train, feed_dict={x:batch_x, y_true: batch_y, hold_prob: 0.5})

    # print output every 100 steps
    if i%100 == 0:
        print('Currently on step {}'.format(i))
        print('Accuracy is ')

        matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
        acc = tf.reduce_mean(tf.cast(matches, tf.float32))
        acc_val = sess.run([acc], feed_dict={x: X_test, y_true: y_test, hold_prob: 1.0})
        print(acc_val)
        print('\n')

Currently on step 0
Accuracy is 
[0.1278]


Currently on step 100
Accuracy is 
[0.7702]


Currently on step 200
Accuracy is 
[0.7909]


Currently on step 300
Accuracy is 
[0.802]


Currently on step 400
Accuracy is 
[0.8055]




### To start tensorboard (don't forget to change steps to 1 in order to show only 1 graph)
```python
train_writer = tf.summary.FileWriter('./tensorboard',
                                      sess.graph)
```
```cli
!tensorboard --logdir=tensorboard
```

# Keras

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

batch_size = 256
num_classes = 10
epochs = 50

#input image dimensions
img_rows, img_cols = 28, 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

### Training

In [6]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 31s 656us/step - loss: 0.8535 - acc: 0.6812 - val_loss: 0.5254 - val_acc: 0.8105
Epoch 2/50
48000/48000 [==============================] - 31s 643us/step - loss: 0.5250 - acc: 0.8068 - val_loss: 0.4108 - val_acc: 0.8486
Epoch 3/50
48000/48000 [==============================] - 31s 646us/step - loss: 0.4482 - acc: 0.8354 - val_loss: 0.3540 - val_acc: 0.8733
Epoch 4/50
48000/48000 [==============================] - 30s 617us/step - loss: 0.4005 - acc: 0.8530 - val_loss: 0.3385 - val_acc: 0.8791
Epoch 5/50
48000/48000 [==============================] - 31s 641us/step - loss: 0.3683 - acc: 0.8660 - val_loss: 0.3040 - val_acc: 0.8895
Epoch 6/50
48000/48000 [==============================] - 31s 639us/step - loss: 0.3467 - acc: 0.8735 - val_loss: 0.2913 - val_acc: 0.8956
Epoch 7/50
48000/48000 [==============================] - 31s 636us/step - loss: 0.3335 - acc: 0.8779 - val_loss: 0.